In [2]:
#load libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
#import variables with stored values and store in dictionary
stored_values_df = pd.read_excel("stored_values.xlsx")
stored_values_dict_df = stored_values_df[["Variable_name","Value"]]
stored_values_dict_df = stored_values_dict_df.set_index(['Variable_name'])
stored_values_dict = stored_values_dict_df.to_dict()
stored_values_dict = stored_values_dict['Value']
stored = stored_values_dict

In [3]:
#import variables with static values and store in dictionary
static_values_df = pd.read_excel("static_values.xlsx")
static_values_dict_df = static_values_df[["Variable_name","Value"]]
static_values_dict_df = static_values_dict_df.set_index(['Variable_name'])
static_values_dict = static_values_dict_df.to_dict()
static_values_dict = static_values_dict['Value']
static = static_values_dict

In [4]:
#import daily inputs
daily_inputs = pd.read_excel('daily_inputs.xlsx')

In [5]:
#create empty output df
column_names = ['Date','New_COVID_Cases','Total_COVID_in_Hospital','COVID_cases_in_SNF', 'PUI_in_SNF','Staff_working_with_COVID_or_PUI_patients', 'Cohorting Coefficient', 'Hospitals_full', 'gloves_used','gowns_used', 'N95s_used','eye_pro_used','surg_proc_masks_used']
outputs = pd.DataFrame(columns = column_names)

In [6]:
#determine PPE use for each day in daily inputs and updates outputs df
COVID_sent_to_hospital = 0
day = 0
Hospitals_full = False
Starting_COVID = stored['COVID_in_SNF']
Starting_PUI = stored['PUI_in_SNF']
Hospital_full_count = 0
for i in range(len(daily_inputs)):
    
    #updates Hospital full variable
    if daily_inputs.at[day,'New_COVID_Cases'] > static['COVID_cases_hospitals_full']:
        Hospitals_full = True
    else:
        Hospitals_full = False
    
    #updates COVID and PUI in SNF
    
    if Hospitals_full == False:
        if day > static['LOS_COVID_SNF']:
            stored['COVID_in_SNF'] = stored['COVID_in_SNF'] - (daily_inputs.at[day - int(static['LOS_COVID_SNF']),'New_COVID_in_SNF'] - daily_inputs.at[day - int(static['LOS_COVID_SNF']), 'SNF_COVID_Hospitalized']) + daily_inputs.at[day,'New_COVID_in_SNF'] - daily_inputs.at[day, 'SNF_COVID_Hospitalized']
        elif day == 0:
            stored['COVID_in_SNF'] = stored['COVID_in_SNF']
        else:
            stored['COVID_in_SNF'] = stored['COVID_in_SNF'] + daily_inputs.at[day,'New_COVID_in_SNF'] - daily_inputs.at[day, 'SNF_COVID_Hospitalized'] - (daily_inputs.at[0,'New_COVID_in_SNF'] - daily_inputs.at[0,'Average_Hospitalized_in_first_10_days'])
        Hospial_full_count = 0
        
    if Hospitals_full == True:
        if day > static['LOS_COVID_SNF']:
            if Hospital_full_count < 10:
                stored['COVID_in_SNF'] = stored['COVID_in_SNF'] - (daily_inputs.at[day - int(static['LOS_COVID_SNF']),'New_COVID_in_SNF'] - daily_inputs.at[day - int(static['LOS_COVID_SNF']), 'SNF_COVID_Hospitalized']) + daily_inputs.at[day,'New_COVID_in_SNF']
            else:
                stored['COVID_in_SNF'] = stored['COVID_in_SNF'] - daily_inputs.at[day - int(static['LOS_COVID_SNF']),'New_COVID_in_SNF'] + daily_inputs.at[day,'New_COVID_in_SNF']
        elif day == 0:
            stored['COVID_in_SNF'] = stored['COVID_in_SNF']
        else:
            stored['COVID_in_SNF'] = stored['COVID_in_SNF'] + daily_inputs.at[day,'New_COVID_in_SNF']  - (daily_inputs.at[0,'New_COVID_in_SNF'] - daily_inputs.at[0,'Average_Hospitalized_in_first_10_days'])
    
        Hospital_full_count += 1
        
    if stored['COVID_in_SNF'] > static['Total_MA_SNF_population']:
        stored['COVID_in_SNF'] = static['Total_MA_SNF_population']
    
        
    #updates the stored value of PUI
    stored['PUI_in_SNF'] = stored['PUI_in_SNF'] - (stored['PUI_in_SNF']/static['COVID_test_turnaround_time']) + daily_inputs.at[day,'New_PUI_in_SNF']
    
    if stored['COVID_in_SNF'] + stored['PUI_in_SNF'] > static['Total_MA_SNF_population']:
        stored['PUI_in_SNF'] = static['Total_MA_SNF_population'] - stored['COVID_in_SNF']
    
    
    #updates cohorting coefficient and staff per patient
    
    Cohorting = (stored['PUI_in_SNF'] + stored['COVID_in_SNF'])/ static['Total_MA_SNF_population']
    HCW_RN_per_resident = static['HCW_RN_per_resident'] + ((1-Cohorting)*(1-static['HCW_RN_per_resident']))
    HCW_CNA_per_resident = static['HCW_RN_per_resident'] + ((1-Cohorting)*(1-static['HCW_RN_per_resident']))
    Non_clinical_staff_per_resident = static['Non_clinical_staff_per_resident'] + ((1-Cohorting)*(1-static['Non_clinical_staff_per_resident']))
    
    #updates staff working with COVID and PUI
    
    COVID_PUI = stored['COVID_in_SNF'] + stored['PUI_in_SNF']
    RNs_COVID_PUI = round((HCW_RN_per_resident * COVID_PUI) * static['HCW_shifts_per_day'] )
    CNA_COVID_PUI = round((HCW_CNA_per_resident * COVID_PUI) * static['HCW_shifts_per_day'] )
    HCWs_COVID_PUI = RNs_COVID_PUI + CNA_COVID_PUI
    Non_clinical_COVID_PUI = round((Non_clinical_staff_per_resident * COVID_PUI) * static['Non_clinical_shifts_per_day'] )

    Staff_working_with_COVID_PUI = HCWs_COVID_PUI + Non_clinical_COVID_PUI
    
    if Staff_working_with_COVID_PUI > static['Total_MA_SNF_employee_patient_contact_population']:
        Staff_working_with_COVID_PUI = static['Total_MA_SNF_employee_patient_contact_population']
    
    #Calculates PPE usage
    
    N95s = round((Staff_working_with_COVID_PUI) / static['Shifts_per_N95'])
    Eye_protection = round(Staff_working_with_COVID_PUI / static['Shifts_per_eyepro'])
    Surgical_Procedural_mask = round((static['Total_MA_SNF_employee_population'] / static['Shifts_per_surgical_procedural_mask']) + static['Visitors_per_day'] + static['Total_MA_SNF_population'])
    Gowns = COVID_PUI * static['HCW_daily_visits']
    Gloves = (static['Total_MA_SNF_population']) * static['HCW_daily_visits'] * 2
    
    #updates the outputs
    outputs = outputs.append({'New_COVID_Cases':daily_inputs.at[day, 'New_COVID_Cases'],'Total_COVID_in_Hospital':daily_inputs.at[day, 'COVID_Hospitalized'],'Cohorting Coefficient': Cohorting,'Date': daily_inputs.at[day, 'Date'],'COVID_cases_in_SNF': stored['COVID_in_SNF'], 'PUI_in_SNF': stored['PUI_in_SNF'], 'Staff_working_with_COVID_or_PUI_patients': Staff_working_with_COVID_PUI, 'Hospitals_full': Hospitals_full, 'gloves_used': Gloves,'gowns_used': Gowns, 'N95s_used': N95s,'eye_pro_used': Eye_protection,'surg_proc_masks_used': Surgical_Procedural_mask}, ignore_index = True)
    day += 1                                                                                      

In [17]:
#export outputs to excel
outputs.to_excel('PPE_outputs.xlsx')